# Test Tensorflow-federated (TFF) library

## Test #4 : custom `ClientWork` process

In [1]:
import os
import collections
import nest_asyncio
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff

tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')

nest_asyncio.apply()

print('Tensorflow version : {}'.format(tf.__version__))
print('Tensorflow-federated version : {}'.format(tff.__version__))
print('# GPUs : {}'.format(len(tf.config.list_logical_devices('GPU'))))

tff.federated_computation(lambda: 'Hello, World!')()

2022-11-22 15:48:36.895982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 15:48:36.990599: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-22 15:48:37.014411: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

Tensorflow version : 2.10.0
Tensorflow-federated version : 0.39.0
# GPUs : 1


2022-11-22 15:48:47.511167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 15:48:47.906341: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 371 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5


b'Hello, World!'

In [2]:
import collections
import tensorflow as tf

from models.interfaces.base_fl_model import BaseFLModel
from models.interfaces.base_image_model import BaseImageModel

class MNISTFLClassifier(BaseImageModel, BaseFLModel):
    def __init__(self, input_size = (28, 28, 1), n_labels = 10, ** kwargs):
        self._init_image(input_size = input_size, ** kwargs)
        self._init_fl(** kwargs)
        
        self.n_labels = n_labels
        
        super().__init__(** kwargs)
    
    def _build_model(self):
        super()._build_model(model = {
            'architecture_name' : 'perceptron',
            'input_shape' : self.input_size,
            'units'       : 32,
            'n_dense'     : 1,
            'activation'  : 'relu',
            'drop_rate'   : 0.,
            'bnorm'       : 'never',
            'output_shape' : self.n_labels,
            'final_bias'   : True,
            'final_activation' : 'softmax'
        })
    
    @property
    def output_signature(self):
        return tf.TensorSpec(shape = (None, 1), dtype = tf.int32)
    
    def __str__(self):
        des = super().__str__()
        des += self._str_image()
        des += self._str_fl()
        des += '- # labels : {}\n'.format(self.n_labels)
        return des

    def compile(self, loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'], ** kwargs):
        super().compile(loss = loss, metrics = metrics, ** kwargs)
    
    def preprocess_data(self, data):
        return (
            tf.expand_dims(data['pixels'], axis = -1),
            tf.cast(tf.reshape(data['label'], [-1, 1]), tf.int32)
        )
    
    def get_dataset_config(self, * args, ** kwargs):
        kwargs['batch_before_map'] = True
        return super().get_dataset_config(* args, ** kwargs)
    
    def get_config(self, * args, ** kwargs):
        config = super().get_config(* args, ** kwargs)
        config.update({
            ** self.get_config_image(),
            ** self.get_config_fl(),
            'n_labels' : self.n_labels
        })
        return config
    
    
model = MNISTFLClassifier(nom = 'test_fl_4')
model.summary()
print(model)


========== Summary of model ==========

Model: "multi_layer_perceptron"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_0 (Dense)             (None, 32)                25120     
                                                                 
 activation (Activation)     (None, 32)                0         
                                                                 
 classification_layer (Dense  (None, 10)               330       
 )                                                               
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                                 
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0

In [3]:
emnist_train, emnist_valid = tff.simulation.datasets.emnist.load_data()
print('Dataset length :\n  Train length : {}\n  Valid length : {}'.format(
    len(emnist_train.client_ids), len(emnist_valid.client_ids)
))
print('Data signature : {}'.format(emnist_train.element_type_structure))

Dataset length :
  Train length : 3383
  Valid length : 3383
Data signature : OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


## Model + process initialization

In [4]:
model.compile()
config = model._get_fl_train_config(
    emnist_train, validation_data = emnist_valid, n_train_clients = 10, n_valid_clients = 5, shuffle_size = 0
)

config

{'model_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_model_fn.<locals>.vanilla_model_fn()>,
 'loss_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_loss_fn.<locals>.<lambda>()>,
 'metrics_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_metrics_fn.<locals>.<lambda>()>,
 'server_optimizer_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_optimizer_fn.<locals>.<lambda>()>,
 'client_optimizer_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_optimizer_fn.<locals>.<lambda>()>,
 'reconstruction_optimizer_fn': <function models.interfaces.base_fl_model.BaseFLModel.get_optimizer_fn.<locals>.<lambda>()>,
 'train_ids': ['f1491_42',
  'f3914_30',
  'f0724_37',
  'f1546_05',
  'f3912_15',
  'f1382_07',
  'f2225_86',
  'f0932_44',
  'f1728_02',
  'f2165_54'],
 'valid_ids': ['f1491_42', 'f3914_30', 'f0724_37', 'f1546_05', 'f3912_15'],
 'x': [<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, nam

In [5]:
model_fn = config['model_fn']

@tff.tf_computation
def initial_model_weights_fn():
    return tff.learning.models.ModelWeights.from_model(model_fn())

weights_type = initial_model_weights_fn.type_signature.result
aggregator_factory = tff.aggregators.MeanFactory()

distributor  = tff.learning.templates.build_broadcast_process(weights_type)
client_work  = model.build_client_work(config['model_fn'], config['client_optimizer_fn'])
aggregator   = aggregator_factory.create(weights_type.trainable, tff.TensorType(tf.float32))
finalizer    = tff.learning.templates.build_apply_optimizer_finalizer(config['server_optimizer_fn'], weights_type)

training_process = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
) 

print(distributor.next.type_signature.formatted_representation())
print()
print(client_work.next.type_signature.formatted_representation())
print()
print(aggregator.next.type_signature.formatted_representation())
print()
print(finalizer.next.type_signature.formatted_representation())
print()

print(training_process.initialize.type_signature.formatted_representation())
print(training_process.next.type_signature.formatted_representation())

(<
  state=<>@SERVER,
  value=<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >@SERVER
> -> <
  state=<>@SERVER,
  result=<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >@CLIENTS,
  measurements=<>@SERVER
>)

(<
  state=<>@SERVER,
  model_weights={<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >}@CLIENTS,
  client_dataset_with_id={<
    int32*,
    <
      float32[?,28,28,1],
      int32[?,1]
    >*
  >}@CLIENTS
> -> <
  state=<>@SERVER,
  result={<
    update=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    update_weight=float32
  >}@CLIENTS,
  measurements=<
    train_metrics=<
      sparse_categorical_accuracy=float32,
      loss=float32,
      num_examples=int64,
      num_batches=int64
    >,

## Training and evaluation

In [6]:
train_ids, train_fed_data = config['train_ids'], config['x']
valid_ids, valid_fed_data = config['valid_ids'], config['validation_data']

state  = training_process.initialize()

sample_data = train_fed_data[:5]
sample_ids  = train_ids[:5]

seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in sample_ids]
result = training_process.next(state, list(zip(seq_train_ids, sample_data)))
print(result.metrics)

2022-11-22 15:48:54.393266: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:48:54.393470: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.06566604), ('loss', 2.6932056), ('num_examples', 533), ('num_batches', 36)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.04444445, 0.05454545, 0.06034483, 0.10434783, 0.05882353],
      dtype=float32)), ('loss', array([2.755179 , 2.7103982, 2.6461573, 2.7398257, 2.620559 ],
      dtype=float32)), ('num_examples', array([ 90, 110, 116, 115, 102])), ('num_batches', array([6, 7, 8, 8, 7])), ('id', array([2, 3, 1, 4, 0], dtype=int32))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [7]:
#state  = training_process.initialize()

sample_data = train_fed_data[:1]
sample_ids  = train_ids[:1]

seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in sample_ids]
result = training_process.next(state, list(zip(seq_train_ids, sample_data)))
print(result.metrics)

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.05882353), ('loss', 2.620559), ('num_examples', 102), ('num_batches', 7)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.05882353], dtype=float32)), ('loss', array([2.620559], dtype=float32)), ('num_examples', array([102])), ('num_batches', array([7])), ('id', array([0], dtype=int32))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [9]:
epochs = 5

train_ids, train_fed_data = config['train_ids'], config['x']
valid_ids, valid_fed_data = config['valid_ids'], config['validation_data']

state  = training_process.initialize()

for _ in range(epochs):
    indexes     = np.random.choice(len(train_fed_data), size = 25)
    sample_data = [train_fed_data[idx] for idx in indexes]
    sample_ids  = [train_ids[idx] for idx in indexes]

    seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in sample_ids]
    result = training_process.next(state, list(zip(seq_train_ids, sample_data)))
    state, metrics = result.state, result.metrics
    print(result.metrics)

2022-11-22 15:50:31.248510: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.264398: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.274428: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.277280: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.321588: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.322021: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.322402: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:31.322434: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.07166124), ('loss', 2.7558067), ('num_examples', 2456), ('num_batches', 168)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.08333334, 0.05882353, 0.06956521, 0.04444445, 0.06818182,
       0.04444445, 0.08333334, 0.10434783, 0.06034483, 0.08333334,
       0.06818182, 0.06818182, 0.13      , 0.10434783, 0.08163265,
       0.06034483, 0.05454545, 0.08163265, 0.06034483, 0.04444445,
       0.05454545, 0.06818182, 0.05882353, 0.10434783, 0.04444445],
      dtype=float32)), ('loss', array([2.8469965, 2.620559 , 2.7045653, 2.755179 , 2.9184117, 2.755179 ,
       2.8469965, 2.7398257, 2.6461573, 2.8469965, 2.9184117, 2.9184117,
       2.6834817, 2.7398257, 2.8531463, 2.6461573, 2.7103982, 2.8531463,
       2.6461573, 2.755179 , 2.7103982, 2.9184117, 2.620559 , 2.7398257,
       2.755179 ], dtype=float32)), ('num_examples', arra

2022-11-22 15:50:32.395134: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.395954: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.398750: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.419162: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.444696: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.508372: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.508767: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:32.508854: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.07980255), ('loss', 3.101888), ('num_examples', 2431), ('num_batches', 168)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.06363636, 0.14      , 0.04901961, 0.11363637, 0.14      ,
       0.06944445, 0.04310345, 0.04310345, 0.11304348, 0.11363637,
       0.06944445, 0.04310345, 0.04901961, 0.14      , 0.06944445,
       0.09183674, 0.09183674, 0.04901961, 0.06363636, 0.09183674,
       0.06944445, 0.06944445, 0.09183674, 0.09183674, 0.04310345],
      dtype=float32)), ('loss', array([3.065065 , 3.1307182, 2.8216481, 3.114571 , 3.1307182, 3.2332153,
       3.2468092, 3.2468092, 3.0259469, 3.114571 , 3.2332153, 3.2468092,
       2.8216481, 3.1307182, 3.2332153, 3.0570993, 3.0570993, 2.8216481,
       3.065065 , 3.0570993, 3.2332153, 3.2332153, 3.0570993, 3.0570993,
       3.2468092], dtype=float32)), ('num_examples', array

2022-11-22 15:50:33.579363: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.581370: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.581470: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.582421: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.604064: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.604292: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.604899: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:33.611019: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.09324378), ('loss', 3.521152), ('num_examples', 2531), ('num_batches', 173)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.11      , 0.06122449, 0.11      , 0.10344828, 0.09090909,
       0.09090909, 0.0882353 , 0.10434783, 0.0882353 , 0.11      ,
       0.11      , 0.10434783, 0.11      , 0.10227273, 0.06122449,
       0.10227273, 0.04166667, 0.09090909, 0.04444445, 0.0882353 ,
       0.11      , 0.10434783, 0.09090909, 0.0882353 , 0.10227273],
      dtype=float32)), ('loss', array([3.6842692, 3.4150226, 3.6842692, 3.5835795, 3.5694823, 3.5694823,
       3.2044475, 3.486783 , 3.2044475, 3.6842692, 3.6842692, 3.486783 ,
       3.6842692, 3.5814264, 3.4150226, 3.5814264, 3.9203625, 3.5694823,
       3.4197135, 3.2044475, 3.6842692, 3.486783 , 3.5694823, 3.2044475,
       3.5814264], dtype=float32)), ('num_examples', array

2022-11-22 15:50:34.743556: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.781429: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.783239: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.804585: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.817290: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.818270: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.831248: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:34.835915: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.09233177), ('loss', 4.168894), ('num_examples', 2556), ('num_batches', 176)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.16521738, 0.08695652, 0.06363636, 0.0775862 , 0.10784314,
       0.08695652, 0.06363636, 0.09090909, 0.05      , 0.02777778,
       0.02777778, 0.08695652, 0.0775862 , 0.16521738, 0.09183674,
       0.08695652, 0.16521738, 0.08695652, 0.09090909, 0.08695652,
       0.02777778, 0.10784314, 0.16521738, 0.02777778, 0.09090909],
      dtype=float32)), ('loss', array([3.9367704, 4.06471  , 4.1661496, 4.2371135, 3.7112954, 4.06471  ,
       4.1661496, 4.37812  , 4.359247 , 4.867237 , 4.867237 , 4.06471  ,
       4.2371135, 3.9367704, 4.052873 , 4.06471  , 3.9367704, 4.06471  ,
       4.37812  , 4.06471  , 4.867237 , 3.7112954, 3.9367704, 4.867237 ,
       4.37812  ], dtype=float32)), ('num_examples', array

2022-11-22 15:50:36.114235: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.121365: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.123749: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.129357: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.176882: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.192763: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.194101: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 15:50:36.204415: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', 0.09796748), ('loss', 5.101942), ('num_examples', 2460), ('num_batches', 169)])), ('train_metrics_per_client', OrderedDict([('sparse_categorical_accuracy', array([0.11111111, 0.1       , 0.11304348, 0.10434783, 0.11818182,
       0.11111111, 0.07777778, 0.07777778, 0.07777778, 0.0775862 ,
       0.12244898, 0.1       , 0.0882353 , 0.11111111, 0.11304348,
       0.10434783, 0.10434783, 0.0882353 , 0.0882353 , 0.11111111,
       0.07777778, 0.10434783, 0.0882353 , 0.09090909, 0.0882353 ],
      dtype=float32)), ('loss', array([6.1724434, 5.304197 , 4.781226 , 4.736877 , 4.934802 , 6.1724434,
       5.4446654, 5.4446654, 5.4446654, 5.1820602, 4.729478 , 5.304197 ,
       4.704996 , 6.1724434, 4.781226 , 4.736877 , 4.736877 , 4.704996 ,
       4.704996 , 6.1724434, 5.4446654, 4.736877 , 4.704996 , 5.4329977,
       4.704996 ], dtype=float32)), ('num_examples', array